In [1]:
#import nessary libraries

from bs4 import BeautifulSoup as bs
import requests
import smtplib
import time
import datetime


In [36]:
#connecting to the website

url='https://www.amazon.com/Data-Analyst-T-Shirt-Male-Boss/dp/B09T5JC2XM/ref=sr_1_1?crid=3QE0W4BK4GHFC&keywords=data%2Banalyst%2Bshirt&qid=1683299953&sprefix=data%2Banalyst%2Bs%2Caps%2C689&sr=8-1&th=1&psc=1'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
page=requests.get(url,headers=headers)
soup1=bs(page.content,"html.parser")   #pulling all the html from the website
soup2=bs(soup1.prettify(),"html.parser")   #reading html data with better indendations

title=soup2.find(id='productTitle').get_text()
price=soup2.find(id="corePriceDisplay_desktop_feature_div").get_text()
# print(price)
# print(title)




In [37]:
#cut and strip with slicing required data from the web
price=price.strip()[1:6]  
title=title.strip()

In [ ]:
#creating csv files and storing data in the csv
import csv

headers=['Title','price']
data=[title,price]

with open('Amazon_Web_Scraper_Dataset.csv','w',newline='',encoding='UTF8') as f:
    writer=csv.writer(f)
    writer.writerow(headers)
    writer.writerow(data)







